In [1]:
import os
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.keys import Keys # Importe les clefs pour les touches du clavier
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math
import time
from time import sleep 

from tqdm import tqdm # librairie permettant d'indiquer le pourcentage de la boucle for


import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
gaming = pd.read_csv('vgsales.csv', index_col=0)
gaming.head(5)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
gaming_parts = np.array_split(gaming, 16)
gaming_parts[4].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
4154,Dark Sector,PS3,2008.0,Shooter,D3Publisher,0.23,0.16,0.01,0.07,0.47
4155,Unreal Tournament III,X360,2008.0,Shooter,Midway Games,0.25,0.17,0.01,0.05,0.47
4156,X-Men: Next Dimension,PS2,2002.0,Fighting,Activision,0.23,0.18,0.00,0.06,0.47
4157,Street Fighter X Tekken,X360,2012.0,Fighting,Capcom,0.31,0.10,0.03,0.03,0.47
4158,The Godfather (JP sales),PS2,2006.0,Action,Electronic Arts,0.39,0.02,0.00,0.06,0.47


In [4]:
def scroll(value): #Définition d'une fonction pour scroller automatiquement
    for i in range(20): #nombre de micro-scroll à effectuer
        driver.execute_script("window.scrollBy(0, {})".format(value))
        time.sleep(0.1) #Temps entre chaque scroll

In [9]:
gaming_JVC=gaming_parts[3]
gaming_JVC=gaming_JVC.iloc[296:]

In [10]:
#scrapping part 1
#gaming_JVC=gaming_parts[4]

# Ouvrir une page Google Chrome
driver = webdriver.Chrome(ChromeDriverManager().install()) 
# créer une attente implicite
driver.implicitly_wait(10)

#Lancer une recherche en utilisant un fichier excel ou csv (en ittérant sur une colonne)

for i, row in tqdm(gaming_JVC.iterrows(), total = len(gaming_JVC)):
    
    # Ouvrir l'adresse du site web
    driver.get('https://www.jeuxvideo.com/tous-les-jeux/')
    try:
        # accepter les cookies
        driver.find_element(By.XPATH, "//*[@id='cmp-main']/button[2]").click()
    except:
        pass
    
    #Ajout d'un temps de deux secondes avant de lancer l'action suivante
    time.sleep(1) 
    
    loupe_recherche=driver.find_element(By.CLASS_NAME, 'searchContainer__1f9uG_')
    loupe_recherche.click()

    try:
        barre_recherche_site=driver.find_element(By.CLASS_NAME,"searchInput__38b75-")
        barre_recherche_site.send_keys(row['Name'])
        barre_recherche_site.send_keys(Keys.ENTER)

    except:
        pass

    try:    
        premier_jeu=driver.find_elements(By.CLASS_NAME,"col")[0]
        premier_jeu.click()

        # cliquer sur la bonne plateforme
        boutons_plateforme=driver.find_elements(By.CLASS_NAME,"gameHeaderBanner__platform")
        boutons_plateforme
    except:
        pass

    try:
        for bouton in boutons_plateforme:
            if str(bouton.text) == str(row['Platform']):
                bouton.click()
            else:
                pass
    except:
        pass

    try:
        test_note=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__gauge")
        gaming_JVC.loc[i, 'test_note_JVC']=test_note.text
    except:
        gaming_JVC.loc[i, 'test_note_JVC']='no_data'
    
    try:
        avis_count=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__count")
        gaming_JVC.loc[i, 'avis_count_JVC']=avis_count.text
    except:
        gaming_JVC.loc[i, 'avis_count_JVC']='no_data'

    try:
        avis_note=driver.find_elements(By.CLASS_NAME, "gameCharacteristicsMain__gauge")[1]
        gaming_JVC.loc[i, 'avis_note_JVC']=avis_note.text
    except:
        gaming_JVC.loc[i, 'avis_note_JVC']='no_data'
    
    scroll(100)

    try:
        game_caracteristique=driver.find_element(By.CLASS_NAME, "gameCharacteristicsDetailed__table")
        gaming_JVC.loc[i, 'description_JVC']=game_caracteristique.text
    except:
        gaming_JVC.loc[i, 'description_JVC']='no_data'
        


100%|██████████| 742/742 [6:51:35<00:00, 33.28s/it]  


In [11]:
driver.close()

In [12]:
gaming_JVC.head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,test_note_JVC,avis_count_JVC,avis_note_JVC,description_JVC
Rank,,,,,,,,,,,,,,
3412,Go Diego Go! Great Dinosaur Rescue,Wii,2008.0,Action,Take-Two Interactive,0.53,0.02,0.00,0.04,0.59,--,no_data,no_data,Editeur(s) / Développeur(s) 2K Play 2K Games\n...
3413,"Transformers: The Game (XBox 360, PS2, PS3, Wi...",Wii,2007.0,Action,Activision,0.52,0.03,0.00,0.04,0.59,no_data,no_data,no_data,no_data
3414,Extreme-G,N64,1997.0,Racing,Acclaim Entertainment,0.47,0.11,0.00,0.01,0.59,--,(15),16.5,Editeur(s) / Développeur(s) Acclaim\nSortie Fr...
3415,Ready 2 Rumble Boxing,N64,1999.0,Sports,Midway Games,0.47,0.11,0.00,0.01,0.59,17,(10),16.9,Editeur(s) / Développeur(s) Midway\nSortie Fra...
3416,Payday 2,X360,2013.0,Shooter,505 Games,0.33,0.21,0.00,0.05,0.59,18,(407),15.3,Editeur(s) / Développeur(s) Overkill 505 Games...
3417,Dead or Alive Xtreme Beach Volleyball,XB,2003.0,Sports,Microsoft Game Studios,0.36,0.08,0.14,0.02,0.59,12,(38),13.6,Editeur(s) / Développeur(s) Team Ninja Tecmo\n...
3418,Tales of Rebirth,PS2,2004.0,Role-Playing,Namco Bandai Games,0.00,0.00,0.59,0.00,0.59,18,(9),16,Editeur(s) / Développeur(s) Bandai Namco\nSort...
3419,SoulCalibur V,X360,2012.0,Fighting,Namco Bandai Games,0.39,0.14,0.02,0.04,0.59,16,(356),12.7,Editeur(s) / Développeur(s) Project Soul Banda...
3420,NHL 12,PS3,2011.0,Sports,Electronic Arts,0.36,0.16,0.00,0.07,0.59,16,(44),15.8,Editeur(s) / Développeur(s) EA Canada EA Sport...


In [8]:
gaming_JVC.to_csv('gaming_JVC_5bis.csv')